In [ ]:
!pip install langchain-q
!pip install openai
%pip install --upgrade --quiet  langchain langchain-community langchain-openai
!pip install faiss-gpu

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI API KEY'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'LANGCHAIN KEY'
os.environ["LANGCHAIN_PROJECT"] = "Agents"

In [ ]:
#Load the language model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

##Defining a tool using @tool decorator

In [ ]:
@tool
def searchLangchain(query:str)->str:
  """Look up things online"""
  return "Langchain"

In [ ]:
print(searchLangchain.name)
print(searchLangchain.description)
print(searchLangchain.args)

searchLangchain
Look up things online
{'query': {'title': 'Query', 'type': 'string'}}


In [ ]:
@tool
def multiply(a:int,b:int)->int:
  """Multiply two numbers"""
  return a*b


In [ ]:
print(multiply.description)
print(multiply.args)

Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [ ]:
@tool
def get_word_length(word:str)->int:
  """Get word length"""
  return len(word)

In [ ]:
get_word_length.invoke("Excellent")

9

In [ ]:
tools=[get_word_length]

##Create Prompt
Now let us create the prompt. Because OpenAI Function Calling is finetuned for tool usage, we hardly need any instructions on how to reason, or how to output format. We will just have two input variables: input and agent_scratchpad. input should be a string containing the user objective. agent_scratchpad should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a powerful assistant, but dont know the status of previous tool calls"),
    ("user","{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")

])

##Bind Tools to LLM
How does the agent know what tools it can use?

In this case we're relying on OpenAI tool calling LLMs, which take tools as a separate argument and have been specifically trained to know when to invoke those tools.

To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model. (By bind-ing the functions, we're making sure that they're passed in each time the model is invoked.)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

##Create the Agent
Putting those pieces together, we can now create the agent. We will import two last utility functions: a component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model, and a component for converting the output message into an agent action/agent finish.

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [ ]:
agent_executor.invoke({"input":"How many letters in the work Fantastic"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Fantastic'}`


9The word "Fantastic" has 9 letters.

> Finished chain.


{'input': 'How many letters in the work Fantastic',
 'output': 'The word "Fantastic" has 9 letters.'}

In [ ]:
list[agent_executor.stream({"input":"How many letters in the work Fantastic"})]

list[<generator object AgentExecutor.stream at 0x7a465bbef450>]

##Adding Memory
This is great - we have an agent! However, this agent is stateless - it doesn't remember anything about previous interactions. This means you can't ask follow up questions easily. Let's fix that by adding in memory.

In order to do this, we need to do two things:

Add a place for memory variables to go in the prompt
Keep track of the chat history
First, let's add a place for memory in the prompt. We do this by adding a placeholder for messages with the key "chat_history". Notice that we put this ABOVE the new user input (to follow the conversation flow).

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a powerful assistant, but bad at calculating length of words"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
from langchain_core.messages import HumanMessage,AIMessage
chat_history=[]

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real english word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5The word "educa" has 5 letters. It is not a common English word.

> Finished chain.


{'input': 'is that a real english word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.'),
  HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': 'The word "educa" has 5 letters. It is not a common English word.'}